In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from collections import defaultdict
from IPython.display import clear_output

In [ ]:
import pymysql


class Python2DB():
    def __init__(self):
        self.EF_DB = pymysql.connect(host='220.90.200.176',
                                port=3306,
                                user='multi',
                                passwd='multi!)@(',
                                db='EasyFree',
                                charset='utf8')

        self.cursor = self.EF_DB.cursor()

    def select(self, table, column):
        sql_qr = """
            SELECT {0} FROM {1}
        """.format(column, table)
        self.cursor.execute(sql_qr)
        re = self.cursor.fetchall()
        print(re)

    def insert(self, table, columns, values):
        sql_qr2 = """
            INSERT INTO {0}({1})
            VALUES ({2})
        """.format(table, columns, values)
        self.cursor.execute(sql_qr2)
        self.EF_DB.commit()

    def update(self, table, set_content, where_content):
        sql_qr3 = """
            UPDATE {0}
            SET {1}
            WHERE {2}
        """.format(table, set_content, where_content)
        self.cursor.execute(sql_qr3)
        self.EF_DB.commit()

    def delete(self, table, where_content):
        sql_qr4 = """
            DELETE FROM {0} WHERE {1}
        """.format(table, where_content)
        self.cursor.execute(sql_qr4)
        self.EF_DB.commit()

In [ ]:
response = requests.get("http://emart.ssg.com/")
bs = BeautifulSoup(response.content, "html.parser")

category = bs.select('li.em_total_mn a')

category_ls = []

for i in category:
    category_name = i.text
    category_number = str(i.attrs["href"])[32:42]
    category_ls.append((category_number, category_name))
    

count = 1

db_test = Python2DB()

category_dict = defaultdict(str)

for c_number, c_name in category_ls:
    response = requests.get("http://emart.ssg.com/category/listCategoryItem.ssg?dispCtgId={}".format(c_number))
    bs = BeautifulSoup(response.content, "html.parser")

    category2 = bs.select('li.none_child a')
    if category2:
    
        for j in category2:
            category_dict[j.text] = j.attrs["data-ilparam-value"]
            db_test.insert('Category', 'category_idx,category_name', "{},'{}'".format(count, j.text))
            count += 1
    else:
        category_dict[c_name] = c_number
        db_test.insert('Category', 'category_idx,category_name', "{},'{}'".format(count, c_name))
        count += 1
        
    time.sleep(10)
    
    clear_output(wait=True)
    print('{}% 진행중...'.format(int(count/len(category_ls)*100)))

In [ ]:
col_dict_df = pd.Series(category_dict).to_frame('new_col').reset_index()

In [ ]:
col_dict_df.columns = ['category_name', 'category_number']

In [ ]:
df_emart_category = pd.DataFrame(col_dict_df, columns = ['category_number', 'category_name'])

In [ ]:
df_emart_category

In [ ]:
df_emart_category.to_csv("./emart_category.csv", encoding = "utf-8")